In [80]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [8]:
try:
    bakehaven_df = pd.read_csv("bakehaven.csv")
    print("Data extracted successfully")

except Exception as e:
    print(f"Something went very wrong: {e}")


bakehaven_df.head()

Data extracted successfully


,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [23]:
bakehaven_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

## Data Cleaning

In [24]:
bakehaven_df = bakehaven_df.drop_duplicates()

In [ ]:
numeric_cols = bakehaven_df.select_dtypes(["float64", "int64"]).columns

for col in numeric_cols:
    bakehaven_df = bakehaven_df.fillna({col: bakehaven_df[col].mean()})

In [ ]:
string_cols = bakehaven_df.select_dtypes(["object"]).columns

for col in string_cols:
    bakehaven_df = bakehaven_df.fillna({col: "Unknown"})

In [ ]:
# Proper renaming of columns
bakehaven_df = bakehaven_df.rename(columns={
    "Date": "date",
    "ProductName": "product_name",
    "Quantity": "quantity",
    "UnitPrice": "unit_price",
    "StoreLocation": "store_location",	
    "PaymentType": "payment_type",
    "PromotionApplied": "promotion_applied",	
    "Weather":	"weather",
    "Temperature": "temperature",	
    "StaffPerformanceRating": "Staff_performance_rating",	
    "CustomerFeedback": "customer_feedback",
    "DeliveryTime_min": "delivery_time_min",
    "OrderType": "order_type",
    "CustomerName": "customer_name",
    "CustomerAddress": "customer_address",
    "Customer_PhoneNumber": "customer_phone_number",
    "CustomerEmail": "customer_email",
    "Staff_Name":	"staff_name",
    "Staff_Email" : "staff_email",
    "DayOfWeek": "day_of_week",
    "TotalSales": "total_sales"
})

bakehaven_df

,date,product_name,quantity,unit_price,store_location,payment_type,promotion_applied,weather,temperature,Staff_performance_rating,...,delivery_time_min,order_type,customer_name,customer_address,customer_phone_number,customer_email,staff_name,staff_email,day_of_week,total_sales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-02-11 11:00:00,Vanilla Cake,2,11.224568,North,Credit Card,True,Rainy,28.787946,Poor,...,29,In Store,Christopher Joyce,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com,Lauren Hines,robertmaxwell@example.com,Saturday,22.449136
996,2023-02-11 12:00:00,Carrot Cake,3,15.691665,East,Credit Card,False,Cloudy,34.641175,Good,...,32,In Store,Rebecca Hines,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com,Roy Garcia,johnsonangela@example.org,Saturday,47.074995
997,2023-02-11 13:00:00,Cheesecake,3,10.793180,West,Cash,True,Sunny,27.984565,Average,...,43,Phone Order,Patricia Turner,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org,Bruce Chung,lori61@example.org,Saturday,32.379540
998,2023-02-11 14:00:00,Cheesecake,5,13.827574,East,Online,True,Rainy,33.579827,Poor,...,52,Online Order,Kathryn Nichols,"7355 Donna Shoals Apt. 887\r\nLake Kevinmouth,...",+1-766-493-7034,mary38@example.org,Leslie Carter,zreed@example.com,Saturday,69.137870


In [59]:
bakehaven_df = bakehaven_df.rename(columns={
    "Staff_performance_rating" : "staff_performance_rating"
})

## Data Transformation

In [46]:
products = bakehaven_df[["product_name"]].drop_duplicates().reset_index(drop=True)
products.index.name = "product_id"
products = products.reset_index()
products["product_id"] += 1

products

,product_id,product_name
0,1,Vanilla Cake
1,2,Red Velvet Cake
2,3,Chocolate Cake
3,4,Carrot Cake
4,5,Pizza Pepperoni
5,6,Pizza Margherita
6,7,Pizza Beef Double decker
7,8,Cheesecake
8,9,Fruit Cake


In [56]:
customers = bakehaven_df[["customer_name", "customer_address", "customer_phone_number", "customer_email"]].drop_duplicates().reset_index(drop=True)

customers.index.name = "customer_id"
customers = customers.reset_index()
customers["customer_id"] += 1


customers

,customer_id,customer_name,customer_address,customer_phone_number,customer_email
0,1,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,2,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,3,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,4,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,5,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net
...,...,...,...,...,...
995,996,Christopher Joyce,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com
996,997,Rebecca Hines,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com
997,998,Patricia Turner,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org
998,999,Kathryn Nichols,"7355 Donna Shoals Apt. 887\r\nLake Kevinmouth,...",+1-766-493-7034,mary38@example.org


In [65]:
staff = bakehaven_df[["staff_name", "staff_email", "staff_performance_rating"]].drop_duplicates().reset_index(drop=True)

staff.index.name ="staff_id"
staff = staff.reset_index()
staff["staff_id"] += 1


staff

,staff_id,staff_name,staff_email,staff_performance_rating
0,1,John Bridges,pdavidson@example.com,Poor
1,2,Sarah Bentley,ajohnson@example.net,Average
2,3,Connie Cervantes,michele29@example.net,Excellent
3,4,Jessica Stewart,xwilson@example.org,Poor
4,5,Cheryl Carpenter,christine96@example.org,Good
...,...,...,...,...
995,996,Lauren Hines,robertmaxwell@example.com,Poor
996,997,Roy Garcia,johnsonangela@example.org,Good
997,998,Bruce Chung,lori61@example.org,Average
998,999,Leslie Carter,zreed@example.com,Poor


In [71]:
transactions = bakehaven_df.merge(products, on=["product_name"], how="left") \
                           .merge(customers, on=["customer_name", "customer_address", "customer_phone_number", "customer_email"], how="left") \
                           .merge(staff, on=["staff_name", "staff_email", "staff_performance_rating"], how="left") 

transactions.index.name = "transaction_id"
transactions = transactions.reset_index()
transactions["transaction_id"] += 1

transactions = transactions[["transaction_id", "product_id",	"customer_id", "staff_id", "date", "quantity", \
                            "unit_price",	"store_location",	"payment_type",	"promotion_applied",	"weather", \
                            "temperature", "customer_feedback", "delivery_time_min", "order_type", "day_of_week", "total_sales" \
                            ]]

transactions

,transaction_id,product_id,customer_id,staff_id,date,quantity,unit_price,store_location,payment_type,promotion_applied,weather,temperature,customer_feedback,delivery_time_min,order_type,day_of_week,total_sales
0,1,1,1,1,2023-01-01 00:00:00,2,12.532304,South,Cash,True,Rainy,20.654914,Neutral,30,In Store,Sunday,25.064608
1,2,2,2,2,2023-01-01 01:00:00,1,7.083070,South,Cash,False,Rainy,23.549497,Positive,33,In Store,Sunday,7.083070
2,3,3,3,3,2023-01-01 02:00:00,5,6.736064,North,Cash,True,Rainy,27.154342,Unknown,43,Phone Order,Sunday,33.680321
3,4,4,4,4,2023-01-01 03:00:00,2,7.314823,North,Cash,False,Cloudy,20.137483,Positive,32,Online Order,Sunday,14.629647
4,5,5,5,5,2023-01-01 04:00:00,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Neutral,58,In Store,Sunday,7.577727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,1,996,996,2023-02-11 11:00:00,2,11.224568,North,Credit Card,True,Rainy,28.787946,Negative,29,In Store,Saturday,22.449136
996,997,4,997,997,2023-02-11 12:00:00,3,15.691665,East,Credit Card,False,Cloudy,34.641175,Positive,32,In Store,Saturday,47.074995
997,998,8,998,998,2023-02-11 13:00:00,3,10.793180,West,Cash,True,Sunny,27.984565,Unknown,43,Phone Order,Saturday,32.379540
998,999,8,999,999,2023-02-11 14:00:00,5,13.827574,East,Online,True,Rainy,33.579827,Positive,52,Online Order,Saturday,69.137870


In [ ]:
# Save data as CSV files

bakehaven_df.to_csv("data/interim/bakehaven_cleaned.csv", index=False)

transactions.to_csv("data/processed/transactions.csv", index=False)
products.to_csv("data/processed/products.csv", index=False)
customers.to_csv("data/processed/customers.csv", index=False)
staff.to_csv("data/processed/staff.csv", index=False)   

In [72]:
bakehaven_df

,date,product_name,quantity,unit_price,store_location,payment_type,promotion_applied,weather,temperature,staff_performance_rating,...,delivery_time_min,order_type,customer_name,customer_address,customer_phone_number,customer_email,staff_name,staff_email,day_of_week,total_sales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-02-11 11:00:00,Vanilla Cake,2,11.224568,North,Credit Card,True,Rainy,28.787946,Poor,...,29,In Store,Christopher Joyce,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com,Lauren Hines,robertmaxwell@example.com,Saturday,22.449136
996,2023-02-11 12:00:00,Carrot Cake,3,15.691665,East,Credit Card,False,Cloudy,34.641175,Good,...,32,In Store,Rebecca Hines,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com,Roy Garcia,johnsonangela@example.org,Saturday,47.074995
997,2023-02-11 13:00:00,Cheesecake,3,10.793180,West,Cash,True,Sunny,27.984565,Average,...,43,Phone Order,Patricia Turner,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org,Bruce Chung,lori61@example.org,Saturday,32.379540
998,2023-02-11 14:00:00,Cheesecake,5,13.827574,East,Online,True,Rainy,33.579827,Poor,...,52,Online Order,Kathryn Nichols,"7355 Donna Shoals Apt. 887\r\nLake Kevinmouth,...",+1-766-493-7034,mary38@example.org,Leslie Carter,zreed@example.com,Saturday,69.137870


## Data Loading

In [81]:
load_dotenv(override=True)

True

In [82]:
connection_string = os.getenv("CONNECTION_STRING")
container_name= os.getenv("CONTAINER_NAME")

In [ ]:
#  Getting the blob client
blob_client = BlobServiceClient.from_connection_string(connection_string)

# Getting container client
container_client = blob_client.get_container_client(container_name)

# Storing files and blob names in an array
files = [
    (bakehaven_df, "interim/bakehaven_cleaned.csv"),
    (transactions, "processed/transactions.csv"),
    (products, "processed/products.csv"),
    (customers, "processed/customers.csv"),
    (staff, "processed/staff.csv") 
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f"{blob_name} saved successfully to Azure Blob Storage")


    




/interim/bakehaven_cleaned.csv saved successfully to Azure Blob Storage
/processed/transactions.csv saved successfully to Azure Blob Storage
/processed/products.csv saved successfully to Azure Blob Storage
/processed/customers.csv saved successfully to Azure Blob Storage
/processed/staff.csv saved successfully to Azure Blob Storage
